In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , MaxPooling2D, Flatten, Activation, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np 
import time
from keras.optimizers import Adam
from keras import callbacks
from keras.callbacks import ModelCheckpoint
from PIL import Image
import numpy as np
import os

In [ ]:
IMG_SIZE = 224

In [ ]:
def label_img(name):
    """
    Function to define the class of the image according to its name. 
    
    Args:

        name (string) : name of the image in the folder (without extension)
        
    Returns :
        
        array of size two :
                    [1,0] -> Twitter
                    [0,1] -> Facebook
    
    """
    word_label = name[-5]
    if word_label == '0': return np.array([1, 0]) #Twitter
    elif word_label == '1' : return np.array([0, 1]) #Facebook

In [ ]:
def load_data(DIR):
    """
    Loading of images
    
    Args :
        DIR : path to de folder containing images
        
    Returns :
        
        List containing for each position, the image matrix and its associated label vector
    
    """
    data = []
    for img in os.listdir(DIR):
        label = label_img(img)
        path = os.path.join(DIR, img)
        if "DS_Store" not in path:
            img = Image.open(path)
            img=img.convert("RGB")
            img = img.resize((IMG_SIZE, IMG_SIZE))
            data.append([np.array(img), label])
    np.random.shuffle(data)
    return data

In [ ]:
def generate_model():
    
    model = Sequential()
    model.add(Conv2D(32,3,padding="same", activation="relu",kernel_initializer='he_normal', input_shape=(224,224,3)))
    model.add(MaxPool2D())

    model.add(Conv2D(32, 3, padding="same", kernel_initializer='he_normal',activation="relu"))
    model.add(MaxPool2D())

    model.add(Conv2D(64, 3, padding="same", activation="relu"))
    model.add(MaxPool2D())
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128,activation="relu"))
    model.add(Dense(2, activation="softmax"))

    return model

In [ ]:
# Training


opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)

for noise in range(0,100,10):
    for batch in range(1,6):
        Train_DIR='path to principal training dir'+str(p)+'/'
        train_data = load_data(Train_DIR)
        trainImages = np.array([i[0] for i in train_data]).reshape(-1,IMG_SIZE, IMG_SIZE, 3)
        trainLabels = np.array([i[1] for i in train_data])
        trainImages=np.float_(trainImages)
        trainLabels=np.float_(trainLabels)
        model=generate_model()
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    
    
        checkpoint_name = 'path to checkpoint dir'+str(noise)+'_b='+str(batch)+'-{epoch:03d}.hdf5' 
        checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
        callbacks_list = [checkpoint]
    
    
    
        gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
        session = tf.compat.v1.InteractiveSession(config= tf.compat.v1.ConfigProto(gpu_options=gpu_options))
        with tf.device("/gpu:0"):
    
            start = time.time() 
            model.fit(trainImages, trainLabels, validation_split=0.2, epochs=100,verbose=1,callbacks=callbacks_list,batch_size=batch, shuffle=True) #save history of learning
            session.close() 

In [ ]:
# Test

Test_DIR='path to test dir'
test_data = load_data(Test_DIR)
testImages = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
testLabels = np.array([i[1] for i in test_data])


with tf.device("/cpu:0"):
    model=generate_model()
    wights_file = 'Path to weight file .hdf5'
    model.load_weights(wights_file) # chargement du modèle
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    loss, acc = model.evaluate(testImages, testLabels, verbose = 1)
    print("accuracy : ",acc*100)